In [1]:
import pandas as pd
import torch

In [9]:
!pip install gensim==4.1

     |████████████████████████████████| 24.0 MB 65.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
%pip install vncorenlp

!git clone https://github.com/vncorenlp/VnCoreNLP.git

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 895 kB 50.1 MB/s 
     |████████████████████████████████| 6.5 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 2.6 MB 4.2 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=e23c86eba12259b4c95128da4448e73b47b1cbbc78fcc399e151f1296dbc5501
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 215 (delta 0), reused 0 (delta 0), pack-

In [11]:
train_data_csv = "reviews-ecommerce.csv"
dataset = pd.read_csv(train_data_csv)
dataset

,comment,label,rate,Unnamed: 3
0,Áo bao đẹp ạ!!,POS,5,NaN
1,Tuyệt vời !,POS,5,NaN
2,2day ao khong giong trong.,NEG,1,NaN
3,"Mùi thơm,bôi lên da mềm da.",POS,5,NaN
4,"Vải đẹp, dày dặn.",POS,5,NaN
...,...,...,...,...
31455,Không đáng tiền.,NEG,1,NaN
31456,Quần rất đẹp.,POS,5,NaN
31457,Hàng đẹp đúng giá tiền.,POS,5,NaN
31458,Chất vải khá ổn.,POS,4,NaN


In [12]:
# Perform preprocessing
import re

# TODO use a teen code list
rep = {"k": "không", "ko": "không", "kh": "không", "đc": "được", "dc": "được", "a": "anh", "e": "em", "v": "vậy", "vs": "với", "m": "mình"} # define desired replacements here

# use these three lines to do the replacement
rep_ = dict((r"\b{}\b".format(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep_.keys()), flags=re.I)

dataset["comment"] = dataset["comment"].apply(lambda text: pattern.sub(lambda m: rep[re.escape(m.group(0)).lower()], text))

dataset

,comment,label,rate,Unnamed: 3
0,Áo bao đẹp ạ!!,POS,5,NaN
1,Tuyệt vời !,POS,5,NaN
2,2day ao khong giong trong.,NEG,1,NaN
3,"Mùi thơm,bôi lên da mềm da.",POS,5,NaN
4,"Vải đẹp, dày dặn.",POS,5,NaN
...,...,...,...,...
31455,Không đáng tiền.,NEG,1,NaN
31456,Quần rất đẹp.,POS,5,NaN
31457,Hàng đẹp đúng giá tiền.,POS,5,NaN
31458,Chất vải khá ổn.,POS,4,NaN


In [13]:
# STOP-WORDS REMOVAL
import gensim

stopwords = []
with open("vietnamese-stopwords.txt", "r", encoding="utf-8") as stopwords_file:
  stopwords = stopwords_file.readlines()
stopwords = [stopword.strip() for stopword in stopwords]
stopwords = [stopword for stopword in stopwords if stopword != '']

dataset["comment"] = dataset["comment"].apply(lambda text: gensim.parsing.preprocessing.remove_stopwords(text, stopwords=stopwords))
dataset

,comment,label,rate,Unnamed: 3
0,Áo bao đẹp ạ!!,POS,5,NaN
1,Tuyệt vời !,POS,5,NaN
2,2day ao khong giong trong.,NEG,1,NaN
3,"Mùi thơm,bôi da mềm da.",POS,5,NaN
4,"Vải đẹp, dày dặn.",POS,5,NaN
...,...,...,...,...
31455,Không tiền.,NEG,1,NaN
31456,Quần đẹp.,POS,5,NaN
31457,Hàng đẹp giá tiền.,POS,5,NaN
31458,Chất vải ổn.,POS,4,NaN


In [14]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP(
    "./VnCoreNLP/VnCoreNLP-1.1.1.jar",
    annotators="wseg,pos",
    max_heap_size="-Xmx2g",
)

In [15]:
def segmentize(text):
  text = rdrsegmenter.tokenize(text)
  return [item for sublist in text for item in sublist] #flatten

dataset["comment"] = dataset["comment"].apply(segmentize)
dataset

,comment,label,rate,Unnamed: 3
0,"[Áo, bao, đẹp, ạ, !, !]",POS,5,NaN
1,"[Tuyệt_vời, !]",POS,5,NaN
2,"[2day, ao, khong, giong, trong, .]",NEG,1,NaN
3,"[Mùi, thơm, ,, bôi, da, mềm, da, .]",POS,5,NaN
4,"[Vải, đẹp, ,, dày_dặn, .]",POS,5,NaN
...,...,...,...,...
31455,"[Không, tiền, .]",NEG,1,NaN
31456,"[Quần, đẹp, .]",POS,5,NaN
31457,"[Hàng, đẹp, giá, tiền, .]",POS,5,NaN
31458,"[Chất, vải, ổn, .]",POS,4,NaN


In [16]:
dataset["comment"] = dataset["comment"].apply(lambda tokens: [token for token in tokens if len(token) > 1])
dataset

,comment,label,rate,Unnamed: 3
0,"[Áo, bao, đẹp]",POS,5,NaN
1,[Tuyệt_vời],POS,5,NaN
2,"[2day, ao, khong, giong, trong]",NEG,1,NaN
3,"[Mùi, thơm, bôi, da, mềm, da]",POS,5,NaN
4,"[Vải, đẹp, dày_dặn]",POS,5,NaN
...,...,...,...,...
31455,"[Không, tiền]",NEG,1,NaN
31456,"[Quần, đẹp]",POS,5,NaN
31457,"[Hàng, đẹp, giá, tiền]",POS,5,NaN
31458,"[Chất, vải, ổn]",POS,4,NaN


In [31]:
# TRAIN word2vec
len(dataset["comment"].values)
w2v_model = gensim.models.Word2Vec(pd.Series(dataset['comment']).values, min_count = 1, vector_size = 768, workers = 2, window = 5, sg = 0)
w2v_model.save("word2vec_ecomm_reviews.gensim.model")

In [33]:
# LOAD pretrained word2vec model
import numpy as np

w2v_model = gensim.models.Word2Vec.load("word2vec_ecomm_reviews.gensim.model")
# Unique ID of the word
print("Index of the word 'mịn':", w2v_model.wv.key_to_index["mịn"])
# Total number of the words 
print(len(w2v_model.wv.key_to_index))
# Print the size of the word2vec vector for one word
print("Length of the vector generated for a word:", len(w2v_model.wv['mịn']))
# Get the mean for the vectors for an example review
print("Print the length after taking average of all word vectors in a sentence:")
print(len(np.mean([w2v_model.wv[token] for token in dataset['comment'][0]], axis=0)))

Index of the word 'mịn': 60
11066
Length of the vector generated for a word: 768
Print the length after taking average of all word vectors in a sentence:
768


In [34]:
# Split dataset into training and test sets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(dataset["comment"].values, dataset["label"].values, test_size=0.15, random_state=42)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=232)

print(train_texts.shape, valid_texts.shape, test_labels.shape)
print(train_texts[:5], train_labels[:5])
print(valid_texts[:5], valid_labels[:5])
print(test_texts[:5], test_labels[:5])

(21392,) (5349,) (4719,)
[list(['Shop', 'giao', 'hàng', 'chớp', 'luôn'])
 list(['Đóng_gói', 'sp', 'cẩn_thận']) list(['Đợi', 'test', 'thử'])
 list(['Shop', 'phục_vụ', 'Rất', 'tiền'])
 list(['Áo', 'hơi', 'mỏng', 'mặc', 'đẹp', 'dây', 'áo', 'hoàn_thiện', 'lắm', 'mong', 'shop', 'khắc_phục'])] ['POS' 'POS' 'NEU' 'NEU' 'POS']
[list(['Chất', 'sản_phẩm', 'tuyệt_vời'])
 list(['Đặt', 'màu', 'vàng', 'màu', 'shop'])
 list(['Tôi', 'hề', 'sản_phẩm', 'này'])
 list(['Áo', 'mỏng', 'ảnh', 'chụp']) list(['Tuyệt'])] ['POS' 'NEG' 'NEG' 'NEG' 'POS']
[list(['Máy', 'chụp', 'ok'])
 list(['Chất', 'sản_phẩm', 'tuyệt_vời', 'Đóng_gói', 'sản_phẩm', 'đẹp', 'chắn', 'Đóng_gói', 'sản_phẩm', 'đẹp', 'chắn', 'Rất', 'tiền', 'Thời_gian', 'giao', 'hàng', 'yêu', 'shop'])
 list(['Chất', 'tam', 'duoc'])
 list(['Hang', 'rât', 'tôt', 'ủng_hộ', 'dài']) list(['Vải', 'đẹp', 'nữa'])] ['POS' 'POS' 'NEG' 'POS' 'POS']


In [102]:
train_vectors = [np.mean([w2v_model.wv[token]for token in tokens], axis=0) for tokens in train_texts]
valid_vectors = [np.mean([w2v_model.wv[token] for token in tokens], axis=0) for tokens in valid_texts]
test_vectors = [np.mean([w2v_model.wv[token] for token in tokens], axis=0) for tokens in test_texts]

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [104]:
unknown_word_embedding = np.zeros(768)
train_vectors = [unknown_word_embedding if isinstance(t, np.float64) else t for t in train_vectors]
valid_vectors = [unknown_word_embedding if isinstance(t, np.float64) else t for t in valid_vectors]
test_vectors = [unknown_word_embedding if isinstance(t, np.float64) else t for t in test_vectors]

In [106]:
import time
from sklearn import svm
import sklearn
# Perform classification with SVM, kernel=linear
classifier_rbf = svm.SVC(kernel='rbf')
t0 = time.time()
classifier_rbf.fit(train_vectors, train_labels)
t1 = time.time()
prediction_linear = classifier_rbf.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))

Training time: 251.089691s; Prediction time: 45.056494s


ValueError: ignored

In [110]:
# results
report = sklearn.metrics.classification_report(test_labels, prediction_linear)
print(report)
# print('positive: ', report['NEG'])
# print('negative: ', report['POS'])
# print('neutral: ', report['NEU'])

              precision    recall  f1-score   support

         NEG       0.52      0.47      0.49      1002
         NEU       0.53      0.03      0.05       745
         POS       0.73      0.94      0.82      2972

    accuracy                           0.69      4719
   macro avg       0.59      0.48      0.46      4719
weighted avg       0.66      0.69      0.63      4719



In [122]:
suc_khoe_comments_df = pd.read_csv("/content/suc-khoe-comments-100-sampled.csv")

def flatten(t):
    return [item for sublist in t for item in sublist]
suc_khoe_comments_df['0'] = suc_khoe_comments_df['0'].apply(lambda text: pattern.sub(lambda m: rep[re.escape(m.group(0)).lower()], text))
suc_khoe_comments_df['0'] = suc_khoe_comments_df['0'].apply(lambda text: gensim.parsing.preprocessing.remove_stopwords(text, stopwords=stopwords))
suc_khoe_comments_df['0'] = suc_khoe_comments_df['0'].apply(segmentize)
suc_khoe_comments_df['0'] = suc_khoe_comments_df['0'].apply(lambda tokens: [token for token in tokens if len(token) > 1])

suc_khoe_comments_segmented = suc_khoe_comments_df['0'].values

In [123]:
suc_khoe_comments_vectors = [np.mean([w2v_model.wv[token] if token in w2v_model.wv else np.zeros(768) for token in tokens], axis=0) for tokens in suc_khoe_comments_segmented]
unknown_word_embedding = np.zeros(768)
suc_khoe_comments_vectors = [unknown_word_embedding if isinstance(t, np.float64) else t for t in suc_khoe_comments_vectors]

In [124]:
prediction_comments = classifier_rbf.predict(suc_khoe_comments_vectors)

In [125]:
report = sklearn.metrics.classification_report(suc_khoe_comments_df["label"].values, prediction_comments)
print(report)

              precision    recall  f1-score   support

         NEG       0.00      0.00      0.00         9
         NEU       0.00      0.00      0.00        67
         POS       0.26      1.00      0.41        24

    accuracy                           0.24       100
   macro avg       0.09      0.33      0.14       100
weighted avg       0.06      0.24      0.10       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [129]:
for i in range(0, 100):
  print(prediction_comments[i], suc_khoe_comments_df.iloc[i][1], suc_khoe_comments_df.iloc[i][0] )

POS NEU ['Mong', 'phép', 'màu', 'con', 'Thương', 'quá', 'cố', 'gắn', 'chiến_đấu', 'nhé', 'mong', 'tất', 'đẹp', 'con']
POS NEU ['Môi_trường', 'sống', 'ô_nhiễm', 'nguyên_nhân', 'bệnh', 'nhất', 'Mỗi', 'dân', 'thức', 'bảo_vệ', 'môi_trường', 'bệnh_tật', 'bớt']
POS NEG ['Trời', 'bệnh', 'covid', 'ngoái', 'suyễn', 'địa', 'khoẻ', 'đi', 'nâng', 'ngực', 'chi']
POS NEU ['Tôi', 'yêu', 'chó', 'hiện', 'nuôi', 'chó.Một', 'thời_gian', 'trưởng_thành', 'chó', 'bắt_đầu', 'cắn', 'vết_thương', 'đôi', 'nặng.Bình', 'thương', 'lời', 'thay_đổi', 'hoàn_toàn', 'đột_ngột', 'cắn', 'thể', 'cơn.Hiện', 'yêu_thương', 'cẩn_thận', 'tiếp_xúc', 'chạm', 'nó.Nói', 'mẹ', 'gia_đình', 'bé', 'bé', 'tiếp_xúc', 'chó', 'gần', 'trong', 'chuồng', 'rào', 'ngăn', 'chó', 'trường_hợp', 'tương', 'xảy', 'ra']
POS NEG ['Nhìn', 'phẫu_thuật', 'ghê']
POS NEG ['Trên', '64', 'tiêm', '65', 'nên', 'thử', 'bao_nhiêu', 'nguòi', '65', 'bệnh', 'nền']
POS POS ['Một', 'cực', 'vui', 'đối_tượng', 'bệnh', 'nặng', 'phù_hợp', 'tiêm', 'vaccine', 'covid', 'Giờ